# Running Parquez pipeline

In [1]:
#!pip install mlrun==0.6.0rc7
!pip show mlrun

Name: mlrun
Version: 0.6.0rc7
Summary: Tracking and config of machine learning runs
Home-page: https://github.com/mlrun/mlrun
Author: Yaron Haviv
Author-email: yaronh@iguazio.com
License: MIT
Location: /User/.pythonlibs/jupyter/lib/python3.7/site-packages
Requires: nest-asyncio, boto3, pydantic, alembic, pyyaml, kubernetes, click, aiohttp, nuclio-jupyter, tabulate, v3io-frames, semver, fastapi, humanfriendly, orjson, urllib3, kfp, dask, pyarrow, sqlalchemy, GitPython, requests, mergedeep, azure-storage-blob, v3io, google-auth, ipython, pandas
Required-by: 


### create the mlrun project 

In [2]:
from os import path, getenv
from mlrun import new_project, mlconf

#project_name = '-'.join(filter(None, ['getting-started-iris', getenv('V3IO_USERNAME', None)]))
project_name = "parquez"
project_path = path.abspath('./')
project = new_project(project_name, project_path)
project.save()
print(f'Project path: {project_path}\nProject name: {project_name}')

Project path: /User/parquez
Project name: parquez


In [3]:
out = mlconf.artifact_path or path.abspath('./data')
# {{run.uid}} will be substituted with the run id, so output will be written to different directoried per run
artifact_path = path.join(out, '{{run.uid}}')
%env PYTHONPATH=./

env: PYTHONPATH=./


### set the project functions

In [9]:
from mlrun.run import new_function
from mlrun import mount_v3io

image_name =  'aviaigz/parquez:0.6.0rc7'
#project.set_function("functions/clean_parquez.py", 'clean', kind='job', image='aviaigz/parquez')
project.set_function("functions/validate_input.py", 'validate', kind='job', image=image_name)
project.set_function("functions/get_table_schema.py", 'get_schema', kind='job', image=image_name)
project.set_function("functions/create_parquet_table.py", 'create_parquet', kind='job', image=image_name)
project.set_function("functions/create_kv_view.py", 'create_kv_view', kind='job', image=image_name)
project.set_function("functions/create_unified_view.py", 'create_unified_view', kind='job', image=image_name)
project.set_function("functions/parquet_add_partition.py", 'parquet_add_partition', kind='job', image=image_name)

spark_job = new_function(kind='spark', command='/User/parquez/functions/kv_to_parquet.py', name='kv_to_parquet') # /User isn't supported at this stage
project.set_function(spark_job)
project.func('kv_to_parquet').with_driver_limits(cpu="1300m")
project.func('kv_to_parquet').with_driver_requests(cpu=1, mem="512m") # gpu_type & gpus=<number_of_gpus> are supported too
project.func('kv_to_parquet').with_executor_limits(cpu="1400m")
project.func('kv_to_parquet').with_executor_requests(cpu=1, mem="512m") # gpu_type & gpus=<number_of_gpus> are supported too
project.func('kv_to_parquet').with_igz_spark() # Adds fuse, daemon & iguazio's jars support
project.func('kv_to_parquet').spec.replicas = 2 # Number of executors
project.func('kv_to_parquet').deploy()# Rebuilds the image with MLRun - This is needed in order to support artifact logging etc. This step is too long (~3 minutes)

delete_kv_partition = new_function(kind='spark', command='/User/parquez/functions/delete_kv_partition.py', name='delete_kv_partition')
project.set_function(delete_kv_partition)
project.func('delete_kv_partition').with_driver_limits(cpu="1300m")
project.func('delete_kv_partition').with_driver_requests(cpu=1, mem="512m") # gpu_type & gpus=<number_of_gpus> are supported too
project.func('delete_kv_partition').with_executor_limits(cpu="1400m")
project.func('delete_kv_partition').with_executor_requests(cpu=1, mem="512m") # gpu_type & gpus=<number_of_gpus> are supported too
project.func('delete_kv_partition').with_igz_spark() # Adds fuse, daemon & iguazio's jars support
project.func('delete_kv_partition').spec.replicas = 2 # Number of executors
project.func('delete_kv_partition').deploy()

project.set_function("functions/run_scheduler.py", 'run_scheduler', kind='job', image=image_name)

> 2021-01-03 15:04:50,486 [info] running build to add mlrun package, set with_mlrun=False to skip if its already in the image
> 2021-01-03 15:04:50,492 [info] starting remote build, image: .mlrun/func-parquez-kv_to_parquet-latest
INFO[0000] Retrieving image manifest gcr.io/iguazio/spark-app:3.0_b5963_20201228195115 
INFO[0000] Retrieving image manifest gcr.io/iguazio/spark-app:3.0_b5963_20201228195115 
INFO[0000] Built cross stage deps: map[]                
INFO[0000] Retrieving image manifest gcr.io/iguazio/spark-app:3.0_b5963_20201228195115 
INFO[0001] Retrieving image manifest gcr.io/iguazio/spark-app:3.0_b5963_20201228195115 
INFO[0001] Executing 0 build triggers                   
INFO[0001] Unpacking rootfs as cmd RUN pip install mlrun==0.6.0-rc7 requires it. 
INFO[0047] RUN pip install mlrun==0.6.0-rc7             
INFO[0047] Taking snapshot of full filesystem...        
INFO[0054] cmd: /bin/sh                                 
INFO[0054] args: [-c pip install mlrun==0.6.0-rc7] 

<a id="gs-step-create-n-run-ml-pipeline"></a>
## Create and Run a Fully Automated ML Pipeline

You're now ready to create a full ML pipeline.
This is done by using [Kubeflow Pipelines](https://www.kubeflow.org/docs/pipelines/overview/pipelines-overview/), which is integrated into the Iguazio Data Science Platform.
Kubeflow Pipelines is an open-source framework for building and deploying portable, scalable machine-learning workflows based on Docker containers.
MLRun leverages this framework to take your existing code and deploy it as steps in the pipeline.

In [5]:
%%writefile {path.join(project_path, 'workflow.py')}

from kfp import dsl
from mlrun import mount_v3io
from os import path
import os

V3IO_ACCESS_KEY = os.environ['V3IO_ACCESS_KEY']
V3IO_USERNAME = os.getenv('V3IO_USERNAME')

funcs = {}
project_path = path.abspath('./')
parquez_params = {'view_name':'view_name'
         ,'partition_by':'h'
         ,'partition_interval':'1h'
         ,'real_time_window':'1d'
         ,'historical_retention':'7d'
         ,'real_time_table_name':'faker'
         ,'config_path':'/User/parquez/config/parquez.ini'
         ,'user_name':V3IO_USERNAME
         ,'access_key':V3IO_ACCESS_KEY          
         ,'project_path': project_path
         ,'shell_pod_name' : 'parquez-shell'       }


# Configure function resources and local settings
def init_functions(functions: dict, project=None, secrets=None):
    project_path = path.abspath('./')
    for f in functions.values():
        f.apply(mount_v3io())
        f.set_env('PYTHONPATH', project_path)
        f.spec.artifact_path = 'User/artifacts'
        f.spec.service_account='mlrun-api'
        
        
# Create a Kubeflow Pipelines pipeline
@dsl.pipeline(
    name = "parquez-pipeline",
    description = "parquez description"
)
def kfpipeline():
    
#     # clean the tables
#     clean = funcs['clean'].as_step(
#         name="clean",
#         params=parquez_params,
#         outputs=['clean']
#     )
    
    # Ingest the data set
    validate = funcs['validate'].as_step(
        name="validate",
        params=parquez_params,
#         inputs={'table': clean.outputs},
        outputs=['validate']
    )
    
    # Analyze the dataset
    schema = funcs['get_schema'].as_step(
        name="get_schema",
        params = parquez_params,
        inputs={'table': validate.outputs},                       
        outputs=['schema']
    )
    
    parquet = funcs["create_parquet"].as_step(
        name="create_parquet",
        params=parquez_params,
        inputs={"table": schema.outputs['schema']},
        outputs=['create_parquet']
    )
    
    kv_view = funcs["create_kv_view"].as_step(
        name="create_kv_view",
        params=parquez_params,
        inputs={'table': parquet.outputs},
        outputs=['kv_view']
    )
    
    unified_view = funcs["create_unified_view"].as_step(
        name="create_unified_view",
        params=parquez_params,
        inputs={'table': kv_view.outputs},
        outputs=['unified_view']
    )
    
    unified_view = funcs["run_scheduler"].as_step(
        name="run_scheduler",
        params=parquez_params,
        inputs={'table': unified_view.outputs},
        outputs=['run_scheduler']
    )    

Overwriting /User/parquez/workflow.py


<a id="gs-register-workflow"></a>
#### Register the Workflow

Use the `set_workflow` MLRun project method to register your workflow with MLRun.
The following code sets the `name` parameter to the selected workflow name ("main") and the `code` parameter to the name of the workflow file that is found in your project directory (**workflow.py**).

In [6]:
# Register the workflow file as "main"
project.set_workflow('main', 'workflow.py')

In [6]:
project.save()

In [8]:
run_id = project.run(
    'main',
    arguments={}, 
    
    artifact_path=path.abspath(path.join('pipeline','{{workflow.uid}}'),
    
                              )
    ,dirty=True)

> 2020-12-28 17:19:10,767 [info] using in-cluster config.


> 2020-12-28 17:19:11,438 [info] Pipeline run id=bd2a4360-e983-47c2-b14f-b3d64023c861, check UI or DB for progress


In [9]:
from mlrun import get_run_db
get_run_db().list_schedules('parquez')

SchedulesOutput(schedules=[ScheduleOutput(name='run_parquez_interval', kind=<ScheduleKinds.job: 'job'>, scheduled_object={'task': {'spec': {'parameters': {'view_name': 'view_name', 'partition_by': 'h', 'partition_interval': '1h', 'real_time_window': '1d', 'historical_retention': '7d', 'real_time_table_name': 'faker', 'config_path': '/User/parquez/config/parquez.ini', 'user_name': 'avia', 'access_key': '1fee9381-f94b-446e-984f-19519d00f184', 'project_path': '/User/parquez', 'shell_pod_name': 'parquez-shell'}, 'output_path': '/User/artifacts', 'function': 'parquez/run_parquez_interval@869ddaca6a784672c6f7de1ba9aeeb7e191c5d9e', 'secret_sources': [], 'scrape_metrics': False}, 'metadata': {'uid': 'f9afc39311044f018ad34c19c6ebde8c', 'name': 'run_parquez_interval', 'project': 'parquez', 'labels': {'v3io_user': 'avia', 'kind': 'spark', 'owner': 'avia'}, 'iteration': 0}, 'status': {'state': 'created'}}, 'schedule': '0 */1 * * * '}, cron_trigger=ScheduleCronTrigger(year=None, month='*', day='*',